In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
from sklearn.metrics import roc_auc_score
import argparse as ap
from sklearn import preprocessing
import sys
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, Dense, Flatten, MaxPooling2D
from numpy import *
from sklearn import *
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from numpy.random import seed
import time
!pip install ete3
from ete3 import NCBITaxa
import math
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

In [ ]:
import numpy as np
import pandas as pd

dataset_path = "/content/dataset.csv"
metadata_path = "/content/metadata.csv"
level = "Species"
DICT = {}

# Read the dataset line by line to populate DICT
with open(dataset_path, 'r') as file:
    for line in file:
        split = line.strip().split(",")  # Adjust to CSV format

        # Extract the bacterial feature based on the level
        if level == 'Species' and "s__" in split[0]:
            otu = split[0].split("s__")[1]
        elif level == 'Genus' and "g__" in split[0]:
            otu = split[0].split("g__")[1]
        elif "f__" in split[0]:
            otu = split[0].split("f__")[1]
        else:
            # Skip lines that do not match the expected format
            continue

        # Clean up the otu name
        otu = otu.split("_noname")[0].split("_unclassified")[0].replace("_", " ").strip()

        # Convert abundances to floats
        abun = np.array(split[1:], dtype=float)

        # Aggregate data in DICT
        if otu in DICT:
            DICT[otu] = np.add(DICT[otu], abun)
        else:
            DICT[otu] = abun

# Create DataFrame from the dictionary
otus = DICT.keys()
# Convert DICT.values() to a list before stacking
abuns = np.vstack(list(DICT.values())).T
x_train = pd.DataFrame(data=abuns, columns=otus)

# Load and process labels
labels = pd.read_csv(metadata_path, sep=',', header=0)
labels = labels.to_numpy().T



In [ ]:
print(x_train)

     Methanobrevibacter smithii  Methanobrevibacter  \
0                       0.33364             0.00000   
1                       0.49776             0.12802   
2                       0.00000             0.00000   
3                       0.00000             0.00000   
4                       0.49446             0.06786   
..                          ...                 ...   
435                     0.02598             0.00000   
436                     2.78607             0.00000   
437                     2.46789             0.00000   
438                     6.72433             0.07156   
439                     0.00000             0.00000   

     Methanosphaera stadtmanae  Acidobacteriaceae|g  Acidobacteriaceae  \
0                          0.0                                     0.0   
1                          0.0                                     0.0   
2                          0.0                                     0.0   
3                          0.0             

In [ ]:
print(labels[1])
X=x_train
y=labels[1]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 1 1 1 0
 0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0]


In [ ]:
print(X.columns)

Index(['Methanobrevibacter smithii', 'Methanobrevibacter',
       'Methanosphaera stadtmanae', 'Acidobacteriaceae|g  Acidobacteriaceae',
       'Actinomyces graevenitzii', 'Actinomyces odontolyticus',
       'Actinomyces turicensis', 'Varibaculum cambriense',
       'Rothia mucilaginosa', 'Rothia',
       ...
       'Xanthomonas axonopodis', 'Xanthomonas fuscans', 'Mycoplasma bovis',
       'Zunongwangia profunda', 'Enterococcus pallens', 'Rhodopirellula',
       'Vibrio furnissii', 'Bacteroides sp 2 2 4', 'Lysinibacillus fusiformis',
       'Lysinibacillus sphaericus'],
      dtype='object', length=606)


In [ ]:
# Initialize NCBITaxa instance
ncbi = NCBITaxa()

# Extract bacterial names from X columns
raw_name = X.columns.values.tolist()

# Get taxonomic IDs and filter out names without valid NCBI IDs
translated_names = ncbi.get_name_translator(raw_name)
filtered_columns = [name for name in raw_name if name in translated_names]
filtered_ids = [str(translated_names[name][0]) for name in filtered_columns]

# Filter X DataFrame to only include columns with valid taxonomic IDs
X_filtered = X[filtered_columns]
X_filtered.columns = filtered_ids  # Rename columns with NCBI IDs
X = X_filtered
print(filtered_ids)

['2173', '2172', '2317', '55565', '1660', '184870', '43675', '32207', '1680', '1683', '1681', '1686', '216816', '28026', '2702', '446660', '1382', '1383', '74426', '102106', '1118060', '84112', '84111', '471189', '133925', '626934', '376804', '47678', '246787', '626929', '310298', '28111', '674529', '338188', '817', '329854', '204516', '291645', '28116', '310297', '291644', '46506', '818', '820', '371601', '487174', '1099853', '28118', '283168', '823', '328812', '46503', '375288', '28123', '501496', '322095', '281920', '1283313', '454154', '577309', '454155', '28125', '28130', '363265', '214856', '626932', '328813', '28117', '1288121', '328814', '239759', '117563', '1352', '1584', '1613', '97478', '1623', '1246', '33968', '46255', '1358', '1328', '113107', '1302', '102684', '68892', '150055', '1318', '1304', '1308', '1343', '333367', '208479', '358743', '1535', '84026', '29361', '1502', '1512', '33037', '543311', '507750', '33031', '384638', '292800', '106588', '35517', '39485', '39488

In [ ]:
# Also, we can use the Newick obtained file to get the tree by using PhyloT, just like the
# description in our paper
tree = ncbi.get_topology(filtered_ids)
print (tree.get_ascii(attributes=["taxid"]))

order = []
num = 1
for node in tree.traverse(strategy='levelorder'):
    if node.is_leaf():
        order.append(node.name)

postorder = []
num = 1
for node in tree.traverse(strategy='postorder'):
    if node.is_leaf():
        postorder.append(node.name)

temp = []
for i in order:
    if i in X.columns:
        temp.append(i)

order = temp

temp1 = []
for i in postorder:
    if i in X.columns:
        temp1.append(i)

postorder  = temp1

known_Xl=X[order]
known_Xp=X[postorder]

known_Xl.to_csv('knownl.csv')
known_Xp.to_csv('knownp.csv')
# for unknown features, we just arrange the taxa with at least genus levels.


                                        /-172042
                                       |
                                   /32207-43675
                                  |    |
                               /1268    \-2047
                              |   |
                              |    \57493-72000
                              |
                              |     /-110932
                              |    |
                              |-85023-55968
                          /85006   |
                         |    |     \-33877
                         |    |
                         |    |--43668
                         |    |
                         |    |-1696-33889
                         |    |
                         |     \-2039
                         |
                         |          /-29405
                         |     /31957
                         |-85009    \-1744
                         |    |
                         |     \-1839
            

In [ ]:
knownl = pd.read_csv('knownl.csv', index_col=0)
knownp = pd.read_csv('knownp.csv', index_col=0)

In [ ]:
knownl

95818  940557  5076  2317  265488   239935  1096  52022  32067  651822  \
0      0.0     0.0   0.0   0.0     0.0  7.97024   0.0    0.0    0.0     0.0   
1      0.0     0.0   0.0   0.0     0.0  0.00000   0.0    0.0    0.0     0.0   
2      0.0     0.0   0.0   0.0     0.0  0.12113   0.0    0.0    0.0     0.0   
3      0.0     0.0   0.0   0.0     0.0  0.00000   0.0    0.0    0.0     0.0   
4      0.0     0.0   0.0   0.0     0.0  2.85556   0.0    0.0    0.0     0.0   
..     ...     ...   ...   ...     ...      ...   ...    ...    ...     ...   
435    0.0     0.0   0.0   0.0     0.0  0.52140   0.0    0.0    0.0     0.0   
436    0.0     0.0   0.0   0.0     0.0  6.40855   0.0    0.0    0.0     0.0   
437    0.0     0.0   0.0   0.0     0.0  0.23191   0.0    0.0    0.0     0.0   
438    0.0     0.0   0.0   0.0     0.0  0.00849   0.0    0.0    0.0     0.0   
439    0.0     0.0   0.0   0.0     0.0  3.67155   0.0    0.0    0.0     0.0   

     ...      550  158836      548      571      573   244366  366649  76860  \
0    ...  0.00000     0.0  0.00000  0.00000  0.01054  0.00000     0.0    0.0   
1    ...  0.00418     0.0  0.00000  0.00000  0.00000  0.00000     0.0    0.0   
2    ...  0.15789     0.0  0.00041  0.00085  0.23691  0.00127     0.0    0.0   
3    ...  0.00000     0.0  0.00000  0.00000  0.00000  0.00000     0.0    0.0   
4    ...  0.00000     0.0  0.00000  0.00000  0.00000  0.00000     0.0    0.0   
..   ...      ...     ...      ...      ...      ...      ...     ...    ...   
435  ...  0.00000     0.0  0.00000  0.00000  0.00000  0.00000     0.0    0.0   
436  ...  0.00000     0.0  0.00000  0.00000  0.00000  0.00000     0.0    0.0   
437  ...  0.00000     0.0  0.00000  0.00000  0.00000  0.00000     0.0    0.0   
438  ...  0.00000     0.0  0.00000  0.00000  0.00000  0.00000     0.0    0.0   
439  ...  0.00000     0.0  0.00000  0.00000  3.17254  0.00000     0.0    0.0   

        1328  1338  
0    0.00000   0.0  
1    0.00063   0.0  
2    0.04310   0.0  
3    0.00536   0.0  
4    0.00000   0.0  
..       ...   ...  
435  0.00000   0.0  
436  0.00000   0.0  
437  0.00000   0.0  
438  0.00000   0.0  
439  0.00000   0.0  

[440 rows x 452 columns]

In [ ]:
def transform_level(X):
    X = np.array(X)
    raw_dim = X.shape[1]
    img_size = math.ceil(raw_dim ** 0.5)
    new_dim = img_size ** 2

    add_blank = np.zeros((X.shape[0], new_dim - raw_dim))
    new_X = np.hstack((X, add_blank))
    new_X = new_X.reshape(X.shape[0], img_size, -1)
    print(new_X.shape)

    base_log = 4
    new_X = np.log(new_X + 1) / np.log(base_log)

    bins_break = [[0.0065536, np.max(new_X)],
                  [0.0016384, 0.0065536],
                  [0.0004096, 0.0016384],
                  [0.0001024, 0.0004096],
                  [0.0000256, 0.0001024],
                  [0.0000064, 0.0000256],
                  [0.0000016, 0.0000064],
                  [0.0000004, 0.0000016],
                  [0.0000001, 0.0000004],
                  [0, 0.0000001]]

    color_arry_num = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    for windows in range(10):
        index = (new_X >= bins_break[windows][0]) & (new_X < bins_break[windows][1])
        new_X[index] = color_arry_num[windows]

    return new_X

def transform_post(X):
    X = np.array(X)
    raw_dim = X.shape[1]
    img_size = math.ceil(raw_dim ** 0.5)
    new_dim = img_size ** 2

    add_blank = np.zeros((X.shape[0], new_dim - raw_dim))
    new_X = np.hstack((X, add_blank))
    new_X = new_X.reshape(X.shape[0], img_size, -1)

    for img in new_X:
        for line in range(img.shape[0]):
            if line % 2 != 0:
                img[line] = img[line][::-1]
    print(new_X.shape)

    base_log = 4
    new_X = np.log(new_X + 1) / np.log(base_log)

    bins_break = [[0.0065536, np.max(new_X)],
                  [0.0016384, 0.0065536],
                  [0.0004096, 0.0016384],
                  [0.0001024, 0.0004096],
                  [0.0000256, 0.0001024],
                  [0.0000064, 0.0000256],
                  [0.0000016, 0.0000064],
                  [0.0000004, 0.0000016],
                  [0.0000001, 0.0000004],
                  [0, 0.0000001]]

    color_arry_num = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    for windows in range(10):
        index = (new_X >= bins_break[windows][0]) & (new_X < bins_break[windows][1])
        new_X[index] = color_arry_num[windows]
    print(new_X[0])
    return new_X


In [ ]:
# par = read_params(sys.argv)
# file_name = str(par['fn'])
# ts = float(par['ts'])
# rs = int(par['rs'])
# et = str(par['et'])


et='z'
ts = 0.2
rs = 1

if et=='t':
    earlystop = EarlyStopping(monitor='loss',
                              min_delta=1e-4,
                              patience=50,
                              verbose=1)
else:
    earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=1e-4,
                              patience=50,
                              verbose=1)

def train_model(X_train, y_train, X_test, epochs, earlystop, et):
    K.clear_session()

    model = Sequential()
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[1], 1)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    model.add(Dropout(0.5))  # Added dropout for regularization


    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Added dropout to dense layer
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy', 'AUC'])

    if et == 't':
        model.fit(X_train, y_train,
                  batch_size=20,
                  epochs=epochs,
                  callbacks=[earlystop],
                  shuffle=True)
    else:
        model.fit(X_train, y_train,
                  batch_size=20,
                  epochs=epochs,
                  validation_split=0.1,
                  callbacks=[earlystop],
                  shuffle=True)
    model_save_path = "model_CNN_T2D.keras"  # Use the new .keras extension
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    pred = model.predict(X_test)[:, 1]
    return pred

In [ ]:
knownl=transform_level(knownl)
knownp=transform_post(knownp)

knownl= np.expand_dims(knownl, axis=3)
knownp = np.expand_dims(knownp, axis=3)

start = time.time()

new_XL_train, new_XL_test, new_XP_train, new_XP_test, \
y_train, y_test = train_test_split(knownl, knownp, y, test_size=ts, random_state=4489)

y_train = to_categorical(y_train, num_classes=2)

print('++++++++++++++++++++++')
print(new_XL_train.shape)


(440, 22, 22)
(440, 22, 22)
[[1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.1 1.  1.  0.2 1.  1.  0.1 0.1
  0.1 1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  0.1 1.  1.  1.  1.  1.  1.  1.  1.  0.1 1.  1.
  1.  1.  0.1 0.1]
 [0.2 1.  1.  1.  1.  0.1 0.1 0.1 0.2 0.1 0.1 0.1 0.1 1.  1.  0.1 0.1 1.
  1.  1.  1.  0.1]
 [1.  1.  1.  1.  1.  0.1 0.1 1.  1.  1.  1.  1.  0.1 0.1 0.1 1.  0.2 0.1
  0.2 1.  0.1 0.4]
 [0.3 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.2 0.1
  0.2 1.  0.1 1. ]
 [1.  1.  1.  1.  0.2 1.  1.  1.  1.  1.  1.  1.  1.  0.1 1.  1.  0.2 1.
  1.  1.  1.  0.1]
 [1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. 

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import time

ave_auc = []
ave_auc1 = []
ave_auc2 = []
ave_auc12 = []
ave_accuracy = []
ave_accuracy1 = []
ave_accuracy2 = []
ave_accuracy12 = []

repeat_seed = rs

for i in range(repeat_seed):
    K.clear_session()
    seed(i)
    tensorflow.random.set_seed(i)
    epochs = 50

    # Training and prediction for L and P
    predL = train_model(new_XL_train, y_train, new_XL_test, epochs, earlystop, et)
    predP = train_model(new_XP_train, y_train, new_XP_test, epochs, earlystop, et)

    # Averaging the predictions
    pred = (predL + predP) / 2
    pred1 = predL
    pred2 = predP
    pred12 = (predL + predP) / 2

    # Converting test labels and predictions to appropriate types
    y_test = y_test.astype(int)
    pred = pred.astype(float)

    # AUC scores
    auc = roc_auc_score(y_test, pred)
    auc1 = roc_auc_score(y_test, pred1)
    auc2 = roc_auc_score(y_test, pred2)
    auc12 = roc_auc_score(y_test, pred12)

    # Accuracy scores
    acc = accuracy_score(y_test, np.round(pred))
    acc1 = accuracy_score(y_test, np.round(pred1))
    acc2 = accuracy_score(y_test, np.round(pred2))
    acc12 = accuracy_score(y_test, np.round(pred12))

    # Appending AUC and accuracy scores
    ave_auc.append(auc)
    ave_auc1.append(auc1)
    ave_auc2.append(auc2)
    ave_auc12.append(auc12)

    ave_accuracy.append(acc)
    ave_accuracy1.append(acc1)
    ave_accuracy2.append(acc2)
    ave_accuracy12.append(acc12)

# Calculating mean AUC and accuracy for all rounds
mauc = np.mean(ave_auc)
mauc1 = np.mean(ave_auc1)
mauc2 = np.mean(ave_auc2)
mauc12 = np.mean(ave_auc12)

macc = np.mean(ave_accuracy)
macc1 = np.mean(ave_accuracy1)
macc2 = np.mean(ave_accuracy2)
macc12 = np.mean(ave_accuracy12)

# Output the results
print('KnownL Mean AUC :  %.4f' % mauc1)
print('KnownP Mean AUC :  %.4f' % mauc2)
print('EnKnownL Mean AUC :  %.4f' % mauc12)

print('KnownL Mean Accuracy: %.4f' % macc1)
print('KnownP Mean Accuracy: %.4f' % macc2)
print('EnKnownL Mean Accuracy: %.4f' % macc12)

end = time.time()
running_time = end - start

# Displaying the best AUC
print('The best AUC of our EPCNN is: %.5f ' % max([mauc, mauc1, mauc2, mauc12]))

# Displaying total running time
print('All Time cost: %.5f s' % running_time)

# Saving results to a file
path = 'EPCNN_ev.txt'
if os.path.exists(path):
    os.remove(path)
file = open(path, 'a')
file.write('EPCNN Mean AUCs: ' + str(mauc) + "\n")
file.write('KnownL Mean AUCs: ' + str(mauc1) + "\n")
file.write('KnownP Mean AUCs: ' + str(mauc2) + "\n")
file.write('EnKnown Mean AUCs: ' + str(mauc12) + "\n")
file.write('KnownL Mean Accuracy: ' + str(macc1) + "\n")
file.write('KnownP Mean Accuracy: ' + str(macc2) + "\n")
file.write('EnKnown Mean Accuracy: ' + str(macc12) + "\n")
file.write('Time for ' + str(repeat_seed) + ' rounds running: ' + str(running_time) + "s")
file.close()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 101ms/step - AUC: 0.4682 - accuracy: 0.5070 - loss: 0.7203 - val_AUC: 0.5417 - val_accuracy: 0.4722 - val_loss: 0.6956
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - AUC: 0.4908 - accuracy: 0.5106 - loss: 0.7124 - val_AUC: 0.5316 - val_accuracy: 0.4722 - val_loss: 0.6968
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - AUC: 0.5378 - accuracy: 0.5217 - loss: 0.7013 - val_AUC: 0.5548 - val_accuracy: 0.4722 - val_loss: 0.6953
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - AUC: 0.4742 - accuracy: 0.4722 - loss: 0.7112 - val_AUC: 0.5540 - val_accuracy: 0.4722 - val_loss: 0.6930
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - AUC: 0.5029 - accuracy: 0.5218 - loss: 0.7019 - val_AUC: 0.5370 - val_accuracy: 0.4722 - val_loss: 0.6949
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - AUC: 0.4773 - accuracy: 0.4910 - loss: 0.7083 - val_AUC: 0.5455 - val_accuracy: 0.4722 - val_loss: 0.6945
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms